In [ ]:
%load_ext autoreload
%autoreload 2

from jaref_bot.data.http_api import ExchangeManager, BybitRestAPI, GateIORestAPI
from jaref_bot.analysis.backtest.grid_bot import GridBotBacktest

import pandas as pd
import polars as pl
import numpy as np
# pd.options.display.float_format = '{:.2f}'.format
from datetime import datetime, timezone, timedelta

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

import warnings
warnings.filterwarnings("ignore")

from tqdm.notebook import tqdm

from jaref_bot.utils.files import get_saved_coins

In [ ]:
async def get_data(symbol, interval, n_iters, exchange):
    df = await exc_manager.get_candles(symbol=symbol, interval=interval, n_iters=n_iters)
    df = df[f'{exchange}_linear'].sort_index()
    return df.dropna()

In [ ]:
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
exc_manager.add_market("gate_linear", GateIORestAPI('linear'))
coin_information = exc_manager.get_instrument_data()

In [ ]:
coin_information['bybit_linear']['MAGIC_USDT']

In [ ]:
saved_coins = get_saved_coins(data_folder='./data/agg_trades')

#### Отчёт по волатильности монеты

In [ ]:
symbol = 'APT'
exchange = 'bybit'

In [ ]:
exc_manager = ExchangeManager()
if exchange == 'bybit':
    exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))
elif exchange == 'gate':
    exc_manager.add_market("gate_linear", GateIORestAPI('linear'))

start_date = datetime.now().replace(microsecond=0) - timedelta(days=30)
end_date = None

fund_hist = await exc_manager.get_funding_history(symbol=f'{symbol}_USDT', start_date=start_date, limit=500)
fund_hist = fund_hist[0]

fund_sum = float(fund_hist['funding'].sum())
fund_len = fund_hist['funding'].shape[0]

hour_df = await get_data(symbol=f'{symbol}_USDT', interval='1h', n_iters=5, exchange=exchange)
hour_df = hour_df.tail(4320) # Берём последние пол года
hour_df.drop(['Exchange', 'Market_type'], axis=1, inplace=True)
daily_df = await get_data(symbol=f'{symbol}_USDT', interval='1d', n_iters=1, exchange=exchange)
daily_df = daily_df.tail(180) # Берём последние пол года

hour_df['Max_range'] = hour_df['High'] - hour_df['Low']
hour_df['Max_range_perc'] = (hour_df['High'] - hour_df['Low']) / hour_df['Low'] * 100
hour_10 = hour_df[hour_df['Max_range_perc'] > 10].shape[0]
hour_10_perc = 100 * hour_10 / hour_df.shape[0]
hour_atr = hour_df['Max_range'].mean()
hour_atr_perc = hour_df['Max_range_perc'].mean()
hour_mtr_perc = hour_df['Max_range_perc'].max()

daily_df['Max_range'] = daily_df['High'] - daily_df['Low']
daily_df['Max_range_perc'] = (daily_df['High'] - daily_df['Low']) / daily_df['Low'] * 100
days_10 = daily_df[daily_df['Max_range_perc'] > 10].shape[0]
days_10_perc = 100 * days_10 / daily_df.shape[0]
day_atr = daily_df['Max_range'].mean()
day_atr_perc = daily_df['Max_range_perc'].mean()
day_mtr_perc = daily_df['Max_range_perc'].max()

print(f'Монета: {symbol}')
print(f'Суммарный фандинг: {fund_sum * 100:.2f}%, расчётов по фандингу: {fund_len}')
print(f'Фандинг. min: {fund_hist['funding'].min() * 100}%; max: {fund_hist['funding'].max() * 100}%')
print()
print(f'ATR за 1 день: {day_atr:.4f} ({day_atr_perc:.2f}%); за 1 час: {hour_atr:.4f} ({hour_atr_perc:.2f}%)')
print(f'MaxTR за 1 день: {day_mtr_perc:.2f}%; за 1 час: {hour_mtr_perc:.2f}%')
print(f'Колебания цены более 10%. Дни: {days_10} ({days_10_perc:.1f}%); Часы: {hour_10} ({hour_10_perc:.1f}%)')

In [ ]:
# daily_df[daily_df['Max_range_perc'] > 30]

In [ ]:
# Монета: VELO
# Суммарный фандинг: 1.83%, расчётов по фандингу: 90
# Фандинг. min: -0.150915%; max: 0.138957%

# ATR за 1 день: 0.0014 (10.10%); за 1 час: 0.0003 (1.88%)
# MaxTR за 1 день: 53.68%; за 1 час: 19.99%
# Колебания цены более 10%. Дни: 67 (37.2%); Часы: 9 (0.2%)
# =========================
# Монета: ICP
# Суммарный фандинг: 0.78%, расчётов по фандингу: 90
# Фандинг. min: -0.012249%; max: 0.021916%

# ATR за 1 день: 0.3943 (7.23%); за 1 час: 0.0769 (1.36%)
# MaxTR за 1 день: 40.36%; за 1 час: 21.53%
# Колебания цены более 10%. Дни: 30 (16.7%); Часы: 2 (0.0%)
# =========================
# Монета: TON
# Суммарный фандинг: 0.84%, расчётов по фандингу: 180
# Фандинг. min: -0.01417%; max: 0.019304%

# ATR за 1 день: 0.2058 (6.58%); за 1 час: 0.0397 (1.22%)
# MaxTR за 1 день: 42.62%; за 1 час: 25.97%
# Колебания цены более 10%. Дни: 24 (13.3%); Часы: 5 (0.1%)
# =========================
# Монета: MAGIC
# Суммарный фандинг: 0.52%, расчётов по фандингу: 200
# Фандинг. min: -0.016177%; max: 0.01651%

# ATR за 1 день: 0.0196 (12.74%); за 1 час: 0.0035 (2.12%)
# MaxTR за 1 день: 77.28%; за 1 час: 32.21%
# Колебания цены более 10%. Дни: 78 (43.3%); Часы: 43 (1.0%)
# =========================

# Монета: APT
# Суммарный фандинг: 0.39%, расчётов по фандингу: 90
# Фандинг. min: -0.033988%; max: 0.011472%

# ATR за 1 день: 0.4117 (8.13%); за 1 час: 0.0795 (1.51%)
# MaxTR за 1 день: 42.90%; за 1 час: 21.57%
# Колебания цены более 10%. Дни: 40 (22.2%); Часы: 3 (0.1%)

#### Загрузка public_trades из файлов .csv

In [ ]:
# Загрузка исторических public_trades из файлов .csv
import re
import os
data_folder = './data/'
pattern = re.compile(r'^([^_]+)\.csv$', re.IGNORECASE)
coins = set()

for filename in os.listdir(data_folder):
    file_path = os.path.join(data_folder, filename)
    
    # Проверяем, что это файл и соответствует шаблону
    if os.path.isfile(file_path):
        match = pattern.match(filename)
        if match:
            coin_name = match.group(1)
            coins.add(coin_name.upper())  # Для единообразия приводим к верхнему регистру

In [ ]:
trades_df = pd.DataFrame()

for name in sorted(coins):
    tdf = pd.read_csv(f'./data/{name}.csv')
    tdf = tdf.drop(['tickDirection', 'trdMatchID', 'grossValue', 'homeNotional', 'foreignNotional', 'RPI'], axis=1)
    trades_df = pd.concat([trades_df, tdf])

In [ ]:
trades_df['date'] = pd.to_datetime(trades_df['timestamp'], unit='s', utc=True).dt.tz_convert('Europe/Moscow')#.dt.floor('s')

In [ ]:
trades_df.drop('timestamp', axis=1)[['date', 'symbol', 'size', 'price', 'side']].rename(columns={'symbol': 'contract'}
                                ).set_index('date').sort_index()#.to_parquet('./data/magic_grid_bot_trades.parquet')

#### Создание датасета для теста grid bot'а

In [ ]:
# Скачивание public_trades
# exc_manager = ExchangeManager()
# exc_manager.add_market("gate_linear", GateIORestAPI('linear'))

# res = await exc_manager.get_trading_history(symbol='PAXG_USDT', start_date=start_date, limit=1000)
# res = res[0]
# res.drop(['exchange', 'contract'], axis=1).to_parquet(f'./data/{token.lower()}_grid_bot_trades.parquet')

In [ ]:
# VELO
# start_date = datetime(2025, 6, 22, 11, 55, 26, tzinfo=Moscow_TZ)
# end_date = datetime(2025, 7, 11, 20, 55, tzinfo=Moscow_TZ)

# PAXG
# start_date = datetime(2025, 6, 21, 4, 24, 25, tzinfo=Moscow_TZ)
# end_date = datetime(2025, 7, 22, 18, 51, 40, tzinfo=Moscow_TZ)

In [ ]:
token = 'MAGIC'

In [ ]:
exc_manager = ExchangeManager()
exc_manager.add_market("bybit_linear", BybitRestAPI('linear'))

Moscow_TZ = timezone(timedelta(hours=3))
start_date = datetime(2025, 7, 29, 20, 10, 0, tzinfo=Moscow_TZ)
end_date = datetime(2025, 8, 1, 3, 55, tzinfo=Moscow_TZ)

fund_hist = await exc_manager.get_funding_history(symbol=f'{token}_USDT', start_date=start_date, end_date=end_date, limit=300)
fund_hist = fund_hist[0]

In [ ]:
fund_hist.tail(1)

In [ ]:
trades_df = pd.read_parquet(f'./data/{token.lower()}_grid_bot_trades.parquet')

In [ ]:
trades_df.head(2)

In [ ]:
df = trades_df.join(fund_hist, how='outer')
df['funding'] = df['funding'].fillna(0)
df.ffill(inplace=True)
df['size'] = df['size'].astype(float)
df['price'] = df['price'].astype(float)
df['funding'] = df['funding'].astype(float)

df = df.reset_index()

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
# Идеи
# 1. Реализовать реинвестирование прибыли. Можно изначально рассчитать, при каком размере банкролла станет покупаться на 1 контракт больше.

In [ ]:
Moscow_TZ = timezone(timedelta(hours=3))
start_date = datetime(2025, 7, 29, 20, 10, 9, tzinfo=Moscow_TZ)
end_date = datetime(2025, 8, 1, 3, 0, 1, tzinfo=Moscow_TZ)
df = df[(df['date'] > start_date) & (df['date'] < end_date)].reset_index(drop=True)
df = pl.DataFrame(df)

In [ ]:
%%time
results = {}

exchange = 'bybit'
token_name = 'MAGIC'
min_price = 0.118
max_price = 0.2
n_grids = 20
leverage = 2
usdt_amount = 100
ct_val = coin_information['bybit_linear']['MAGIC_USDT']['ct_val']
price_scale = coin_information['bybit_linear']['MAGIC_USDT']['price_scale']
order_size = None
take_profit = None
continue_after_tp = False
close_trades = True

# for n_grids in range(5, 101, 5):
grid_bot = GridBotBacktest(exchange=exchange, token_name=token_name, df=df, mode='long', min_price=min_price, max_price=max_price, n_grids=n_grids, 
                    usdt_amount=usdt_amount, leverage=leverage, price_scale=price_scale, ct_val=ct_val, close_trades=close_trades,
                    market_order_fee=0.00055, limit_order_fee=0.0002, order_size=order_size, take_profit=take_profit, 
                    continue_after_tp=continue_after_tp,
                    verbose=True, verbose_deals=True)
grid_bot.run()



profit = grid_bot.trades_history['pnl'].sum()
grid_profit = grid_bot.trades_history.filter(pl.col('type') == 'trade').select('pnl').sum().item()
n_deals = grid_bot.trades_history.filter(pl.col('type') == 'trade').height
funding = grid_bot.trades_history.filter(pl.col('type') == 'funding').select('pnl').sum().item()
fees = grid_bot.trades_history.filter(pl.col('type') == 'trade').select('open_fee', 'close_fee').sum().sum_horizontal().item()
mean_pnl = grid_bot.trades_history.filter(pl.col('type') == 'trade').select('pnl').mean().item()

results[n_grids] = {'n_deals': n_deals, 'profit': profit, 'grid_profit': grid_profit, 'funding': funding, 'fees': fees, 'mean_pnl': mean_pnl}

print(f'{n_grids=}: {n_deals=}; PnL: {profit:.2f}; grid profit: {grid_profit:.2f}; funding: {funding:.2f}; fees: {fees:.2f}; mean_pnl: {mean_pnl:.2f}')

In [ ]:
# MAGIC
# n_grids=20: n_deals=227; PnL: 24.14; grid profit: 21.44; funding: 2.70; fees: 0.38; mean_pnl: 0.09
# n_grids=65: n_deals=2356; PnL: 22.90; grid profit: 20.38; funding: 2.51; fees: 1.14; mean_pnl: 0.01

In [ ]:
print(grid_bot.open_positions)

In [ ]:
print(grid_bot.grid)

In [ ]:
grid_bot.df

In [ ]:
grid_bot.trades_history.sample(3)

In [ ]:
def create_grid(min_price, max_price, ct_val, price_scale, n_grids):
    """Создаёт уровни сетки с округлением до указанной точности."""
    grid = [min_price]
    price = min_price
    grid_space = round((max_price - min_price) / (n_grids), price_scale)
    
    for i in range(n_grids - 1):
        price = round(price + grid_space, price_scale)
        grid.append(price)
    return grid 

def calculate_order_size(min_price, max_price, usdt_amount, leverage, ct_val, n_grids):
    """Рассчитывает количество контрактов на один ордер."""
    av_price = (min_price + max_price) / 2
    av_price = max_price
    total_contracts = int(usdt_amount * leverage / (av_price * ct_val))
    contracts_per_order = int(total_contracts / (n_grids - 1))
    return contracts_per_order


In [ ]:
usdt_amount = 100
leverage = 2
min_price = 0.0088
max_price = 0.0155
ct_val = 1
price_scale = 6
n_grids = 50

In [ ]:
print(create_grid(min_price, max_price, ct_val, price_scale, n_grids))

In [ ]:
calculate_order_size(min_price, max_price, usdt_amount, leverage, ct_val, n_grids)

In [ ]:
0.015366 /

In [ ]:
200 / 0.01483 / 50

In [ ]:
0.01416 - 0.014026, 0.014294 - 0.01416, 0.014026 - 0.013892

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
x_axis = list(results.keys())
sum_profit_arr = []
grid_profit_arr = []
fee_arr = []
funding_arr = []
mean_pnl_arr = []

for key, value in results.items():
    sum_profit_arr.append(value['profit'])
    grid_profit_arr.append(value['grid_profit'])
    fee_arr.append(value['fees'])
    funding_arr.append(value['funding'])
    mean_pnl_arr.append(value['mean_pnl'])

In [ ]:
plt.figure(figsize=(14, 3))
plt.plot(x_axis, sum_profit_arr, label='Profit')
plt.plot(x_axis, grid_profit_arr, label='Grid Profit')
plt.plot(x_axis, fee_arr, label='Fees')
plt.plot(x_axis, mean_pnl_arr, label='Mean PnL')

plt.xlabel('n_grids')
plt.ylabel('Cumulative PnL')
plt.legend()
plt.show()

In [ ]:
# Прежде чем запускать любого бота, необходимо проверить историю фандинга.